# Análisis de Unión de Filtrado en Pandas

En este notebook analizaremos un tipo de unión conocido como **unión de filtrado**.

Pandas no proporciona soporte directo para realizar uniones de este tipo, pero aprenderemos cómo replicarlas utilizando las herramientas que ofrece la librería.

## ¿Qué es una unión de filtrado?

Una **unión semi** filtra la tabla izquierda, dejando únicamente aquellas observaciones que coinciden con la tabla derecha.
Es decir, es similar a una **unión interna** (`inner join`), ya que sólo se conservan las filas que están presentes en ambas tablas (la intersección).

---

A lo largo del notebook, veremos ejemplos prácticos y cómo implementarlos paso a paso en Pandas.

In [9]:
import pandas as pd
cli = pd.read_csv('../../data/clientes.csv')
ped = pd.read_csv('../../data/pedidos.csv')
# Lo primero que tenemos que hacer es fusionar las dos tablas.
cli_ped = cli.merge(ped,on='id_cliente')

# Imprimimos los primeros resultados
cli_ped.head()

   id_cliente             nombre                 correo
0           1          Ana Pérez    ana.perez@email.com
1           2         Juan López   juan.lopez@email.com
2           3        María Gómez  maria.gomez@email.com
3           4  Ricardo Rodriguez     rich_rod@email.com


,id_cliente,nombre,correo,id_pedido,fecha,total
0,1,Ana Pérez,ana.perez@email.com,101,2024-06-01,150.0
1,1,Ana Pérez,ana.perez@email.com,102,2024-06-03,200.0
2,1,Ana Pérez,ana.perez@email.com,105,2024-06-11,120.0
3,2,Juan López,juan.lopez@email.com,103,2024-06-05,75.5
4,3,María Gómez,maria.gomez@email.com,104,2024-06-10,320.0


In [2]:
# El siguiente paso es  utilizar el metodo isin()
cli['id_cliente'].isin(cli_ped['id_cliente'])

0    True
1    True
2    True
Name: id_cliente, dtype: bool

In [3]:
# Para combinar todo, usamos esta linea para crear un subconjunto de la tabla.

top_ped = cli[cli['id_cliente'].isin(cli_ped['id_cliente'])]
print(top_ped.head())

   id_cliente       nombre                 correo
0           1    Ana Pérez    ana.perez@email.com
1           2   Juan López   juan.lopez@email.com
2           3  María Gómez  maria.gomez@email.com


## ¿Qué es una anti unión?

Una **anti unión** devuelve las observaciones de la tabla izquierda que **no** tienen una observación coincidente en la tabla derecha.
Además, únicamente se incluyen las columnas de la tabla izquierda en el resultado.

En otras palabras, la anti unión permite identificar aquellos registros exclusivos de la primera tabla, excluyendo cualquier coincidencia con la segunda.

---

Más adelante, veremos cómo replicar este comportamiento en Pandas con ejemplos prácticos.

In [33]:
# El indicator = True nos permite ver de donde proviene cada fila.
anti_cli_ped = cli.merge(ped,on='id_cliente',how='left',indicator=True)
anti_cli_ped

,id_cliente,nombre,correo,id_pedido,fecha,total,_merge
0,1,Ana Pérez,ana.perez@email.com,101.0,2024-06-01,150.0,both
1,1,Ana Pérez,ana.perez@email.com,102.0,2024-06-03,200.0,both
2,1,Ana Pérez,ana.perez@email.com,105.0,2024-06-11,120.0,both
3,2,Juan López,juan.lopez@email.com,103.0,2024-06-05,75.5,both
4,3,María Gómez,maria.gomez@email.com,104.0,2024-06-10,320.0,both
5,4,Ricardo Rodriguez,rich_rod@email.com,NaN,NaN,NaN,left_only


In [34]:
# A continuacion utilizaremos el accesor loc y la columna _merge para filtrar las filas que no tienen una coincidencia en la tabla derecha.
anti_cli_ped = anti_cli_ped.loc[anti_cli_ped['_merge'] == 'left_only','id_cliente']
print(anti_cli_ped.head())

5    4
Name: id_cliente, dtype: int64


In [49]:
# En el paso final, utilizaremos el metodo isin() para filtrar las filas con gids en nuestra lista

anti_cli_ped = cli.merge(ped,on='id_cliente',how='left',indicator=True)
list = anti_cli_ped.loc[anti_cli_ped['_merge'] == 'left_only','id_cliente']
# Esto nos da los que no estan en la tabla de pedidos.
non_top = cli[cli['id_cliente'].isin(list)]
print(non_top.head())

   id_cliente             nombre              correo
3           4  Ricardo Rodriguez  rich_rod@email.com
